In [13]:
from __future__ import print_function
import keras
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D


In [14]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layes = [
            keras.layers.Conv2D(filters, 3, strides= strides, padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False),
            keras.layers.BatchNormalization()
        ]
        
        self.skip_layers = []
        if strides>1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 3, strides= strides, padding="same", use_bias=False),
                keras.layers.BatchNormalization()                
            ]
            
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layes:
            Z = layer(Z)
        skip_Z = inputs
        
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            "activation": self.activation,
            "skip_layers": self.activation,
            "main_layers" :self.main_layes
        })
        return config

In [15]:

model = Sequential()


model.add(Conv2D(64,7,strides=2 ,padding='same',input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

prev_filters = 64

for filters in [64]*3 + [128]*4 + [256]*6 +[512]*3:
    strides = 1 if filters==prev_filters else 2
    model.add(ResidualUnit(filters,strides=strides))
    prev_filters = filters
    
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(5, activation="softmax"))

In [16]:
print(model.summary())



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_72 (Conv2D)           (None, 24, 24, 64)        3200      
_________________________________________________________________
batch_normalization_72 (Batc (None, 24, 24, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
residual_unit_32 (ResidualUn (None, 12, 12, 64)        74240     
_________________________________________________________________
residual_unit_33 (ResidualUn (None, 12, 12, 64)        74240     
_________________________________________________________________
residual_unit_34 (ResidualUn (None, 12, 12, 64)       

In [17]:

num_classes = 5
img_rows,img_cols = 48,48
batch_size = 32
train_data_dir = './dataset/train'
validation_data_dir = './dataset/validation'
train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					width_shift_range=0.4,
					height_shift_range=0.4,
					horizontal_flip=True,
					fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)



Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [18]:

checkpoint = ModelCheckpoint('Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                          patience=6,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=6,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(optimizer = 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

history=model.fit(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)


Epoch 1/25
119/755 [===>..........................] - ETA: 16:30 - loss: 1.9328 - accuracy: 0.2568

KeyboardInterrupt: 

In [8]:
z = [64]*3 + [128]*4 + [256]*6 +[512]*3

In [9]:
z

[64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256, 256, 256, 512, 512, 512]

In [ ]:
755/755 [==============================] - 359s 476ms/step - loss: 0.9760 - accuracy: 0.6211 - val_loss: 0.7748 -
val_accuracy: 0.6989